# 개선해야할 점
    1) 한자제거
    2) [사진,포토] 등 불필요한 단어 제거,한자제거
    3) 단어분포도(word distribution)
    4) 불용어처리 가능하게 하기 
    5) 텍스트유사도 분석
    6) 유사도 붙이기 
    7) 유사도 60%넘는거 삭제
    8) 0점처리-건우오빠한테 물어보기
    

1,2,3,4 하기 

# 1.데이터 로드 (Data Load)

In [11]:
import re
import pandas as pd
import numpy as np
import json
#from bs4 import Beautiful Soup
# from nltk.corpus import Beautiful Soup
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer
import konlp
import os
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [12]:
#헤더추가 (title,content,time)
train =pd.read_csv('dataset3000.csv',names=['title','content','time','nan'])
train.head()
train.shape
train.dtypes


title       object
content     object
time        object
nan        float64
dtype: object

In [14]:
# nan 변수버리기
train.drop(['nan'], axis='columns', inplace=True)

KeyError: "['nan'] not found in axis"

In [15]:
train.head()

,title,content,time
0,"'[사진]조여정,'우아한 플라워원피스 입고''",'제24회 춘사영화제는 ‘아시아를 대표하는 영화제로의 도약’을 표방한 제23회에 이...,2019-07-20 16:40
1,'[프로필]국세청 과장급 전보자(7월 22일 자)','조사기획1계장 ▲경주세무서장 ▲국세청 인도네시아 주재관 ▲외교부 파견 ▲서울지방국...,2019-07-20 16:40
2,'[조이HD]춘사영화제 참석한 봉준호 감독-한진원 작가','말레이시아 Finas(말레시아 영화진흥위)의 신임 대표인 Ahmad Idham을 ...,2019-07-20 16:40
3,"'[사진]클라라,'시선 올킬 블루 롱드레스''",'제24회 춘사영화제는 ‘아시아를 대표하는 영화제로의 도약’을 표방한 제23회에 이...,2019-07-20 16:40
4,"'[사진]7개부문 노미네이트 기생충 봉준호 감독,'엄청난 환호 받으며''",'제24회 춘사영화제는 ‘아시아를 대표하는 영화제로의 도약’을 표방한 제23회에 이...,2019-07-20 16:40


# only title

In [5]:
title=list(train['title'])

In [6]:
title

["'[사진]조여정,'우아한 플라워원피스 입고''",
 "'[프로필]국세청 과장급 전보자(7월 22일 자)'",
 "'[조이HD]춘사영화제 참석한 봉준호 감독-한진원 작가'",
 "'[사진]클라라,'시선 올킬 블루 롱드레스''",
 "'[사진]7개부문 노미네이트 기생충 봉준호 감독,'엄청난 환호 받으며''",
 "'[사진]함소원,'우아한 입장''",
 "'[조이HD]엄태구, 지적인 남자야~'",
 "'[조이HD]진기주, 매력적인 미모'",
 "'[조이HD]'기생충' 이정은, 엄마 미소~'",
 "'[조이HD]공명, 춘사영화제 빛내는 미소~'",
 "'[조이HD]'춘사영화제' 조여정, 칸이 반한 미소~'",
 "'[조이HD]'춘사영화제' 문지인, 민트여신~'",
 "'[사진]공명,'훈남 매력''",
 "'무역위, 전기프라이팬 특허권 침해 불공정무역행위 조사 개시'",
 "'[사진]베이비페이스 이성경,'화려한 변신''",
 "'[사진]클라라,'섹시하게''",
 "'[사진]상체 노출 클라라,'이보다 더 아찔할 수 없어''",
 "'[조이HD]조여정, 춘사영화제 여우주연상에 빛나는 자태'",
 "'[조이HD]함소원, 연하 남편을 향한 마음'",
 "'[사진]춘사영화제 참석한 기생충 봉준호 감독'",
 "'[사진]이정은,'영화 팬 여러분 사랑해요''",
 "'[사진]문지인,'섹시 레드립''",
 "'[사진]하지영,'어느덧 여배우''",
 "'[사진]김강현,'춘사영화제 왔어요''",
 "'[사진]공명,'달콤한 하트''",
 "'[조이HD]전여빈, 우아한 블랙 드레스'",
 "'[조이HD]전여빈, 춘사영화제가 주목하는 신예'",
 "'[사진]공명,'젠틀하게''",
 "'[조이HD]문지인, 10년차 베테랑 여배우~'",
 "'[사진]전여빈,'아름다운 미소''",
 "'[조이HD]'춘사영화제' 이성경, 금빛 몸매'",
 "'[조이HD]클라라, 보기만 해도 시원한 블루 드레스'",
 "'[사진]정애연,'단아한 매력''",
 "'[사진]정애연,'춘사영화제 레드카펫 밟으며''",


## 1) [   ] 안에 있는 단어 제거

In [16]:
tep=title

In [8]:
## 정규표현식으로 '[]' 안에 있는거 제거.

In [17]:
#print(re.sub('[사진]', '',tep))

#리스트라서 하나씩
tep1=[]
for i in range(len(tep)) :
    tep1.append(re.sub('[사진]','',tep[i]))

print(tep1[:10])

["'[]조여정,'우아한 플라워원피스 입고''", "'[프로필]국세청 과장급 전보자(7월 22일 자)'", "'[조이HD]춘영화제 참석한 봉준호 감독-한원 작가'", "'[]클라라,'시선 올킬 블루 롱드레스''", "'[]7개부문 노미네이트 기생충 봉준호 감독,'엄청난 환호 받으며''", "'[]함소원,'우아한 입장''", "'[조이HD]엄태구, 지적인 남자야~'", "'[조이HD]기주, 매력적인 미모'", "'[조이HD]'기생충' 이정은, 엄마 미소~'", "'[조이HD]공명, 춘영화제 빛내는 미소~'"]


## 2) 한자변환 (Convert Chinese character)

In [18]:
# replace1=tep.replace('日','일본')

## 3)한글이 아닌 단어 지우기 

In [19]:
#전체제목
# 리스트전체가 아니라 하나씩 빼서 처리해야하기 때문에 for문으로 처리.
#1.한글 및 공백을 제외한 문자를 모두 제거
for i in range(len(title)) :
    title[i] = re.sub("[^가-힣ㄱ-ㅎ ㅏ-ㅣ\\s]","", title[i])

print(title)

['사진조여정우아한 플라워원피스 입고', '프로필국세청 과장급 전보자월 일 자', '조이춘사영화제 참석한 봉준호 감독한진원 작가', '사진클라라시선 올킬 블루 롱드레스', '사진개부문 노미네이트 기생충 봉준호 감독엄청난 환호 받으며', '사진함소원우아한 입장', '조이엄태구 지적인 남자야', '조이진기주 매력적인 미모', '조이기생충 이정은 엄마 미소', '조이공명 춘사영화제 빛내는 미소', '조이춘사영화제 조여정 칸이 반한 미소', '조이춘사영화제 문지인 민트여신', '사진공명훈남 매력', '무역위 전기프라이팬 특허권 침해 불공정무역행위 조사 개시', '사진베이비페이스 이성경화려한 변신', '사진클라라섹시하게', '사진상체 노출 클라라이보다 더 아찔할 수 없어', '조이조여정 춘사영화제 여우주연상에 빛나는 자태', '조이함소원 연하 남편을 향한 마음', '사진춘사영화제 참석한 기생충 봉준호 감독', '사진이정은영화 팬 여러분 사랑해요', '사진문지인섹시 레드립', '사진하지영어느덧 여배우', '사진김강현춘사영화제 왔어요', '사진공명달콤한 하트', '조이전여빈 우아한 블랙 드레스', '조이전여빈 춘사영화제가 주목하는 신예', '사진공명젠틀하게', '조이문지인 년차 베테랑 여배우', '사진전여빈아름다운 미소', '조이춘사영화제 이성경 금빛 몸매', '조이클라라 보기만 해도 시원한 블루 드레스', '사진정애연단아한 매력', '사진정애연춘사영화제 레드카펫 밟으며', '사진주지훈그 만의 시그널', '사진주지훈럭셔리하게', '사진주지훈오늘도 멋짐 폭발', '사진기생충 박명훈춘사영화제 수상 기대해요', '사진한진원 작가봉준호 감독기생충 초대 받았어요', '사진골드 드레스 이성경귀걸이도 금빛 하트', '사진레드카펫 런웨이 이성경모델은 달라도 달라', '사진이성경금빛 각선미', '사진조여정미모 여전해', '사진조여정아기 같은 미소', '사진조여정미소 여신', '사진꽃보다 더 빛나는 조여정의 미모', '사진미소가 아름다운 조여정', '중기 해외진출 지원 발벗고 나선 부산경진

# 2. 형태소 나누기 

In [22]:
from konlpy.tag import Okt

In [23]:
#전체 형태소나누기
#불용어 제거하기 위해 문장 단어 나누기
okt=Okt() 
title_text=[ ] # or title_text=[''] * len(title)
for i in range(len(title)) :
    title_text.append(okt.morphs(title[i],stem=True))

print(title_text[:10])

C:\Users\glass\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


[['사진', '조여정', '우아하다', '플라워', '원피스', '입다'], ['프로필', '국세청', '과장', '급', '전보', '자월', '일', '자'], ['조', '이춘사', '영화제', '참석', '한', '봉준호', '감독', '한', '진원', '작가'], ['사진', '클라라', '시선', '올킬', '블루', '롱', '드레스'], ['사진', '개', '부문', '노미네이트', '기생충', '봉준호', '감독', '엄청나다', '환호', '받다'], ['사진', '함소원', '우아하다', '입장'], ['조이', '엄태구', '지적', '인', '남자', '야'], ['조이진', '기주', '매력', '적', '인', '미모'], ['조이', '기생충', '이정은', '엄마', '미소'], ['조이', '공명', '추다', '영화제', '빛', '내다', '미소']]


## 4) 단어 분포도(Word Distribution)

In [20]:
# 명사만 뽑기

def tokenize(doc):
    return okt.nouns(doc)

In [25]:
sentences_new=tokenize(title)

TypeError: No matching overloads found for kr.lucypark.okt.OktInterface.tokenize(list,java.lang.Boolean,java.lang.Boolean), options are:
	public java.util.List kr.lucypark.okt.OktInterface.tokenize(java.lang.String,java.lang.Boolean,java.lang.Boolean)

	at JPMethod::findOverload(native\common\jp_method.cpp:242)
	at JPMethod::findOverload(native\common\jp_method.cpp:245)
	at JPMethod::invoke(native\common\jp_method.cpp:253)
	at PyJPMethod::__call__(native\python\pyjp_method.cpp:142)


In [1]:
# 폰트경로:'C:/Users/glass/Downloads/gulim.ttc'

In [ ]:
# tokens = [j for i in sentences_new for j in i]


# import nltk

# text = nltk.Text(tokens) 


# import matplotlib.pyplot as plt
# from matplotlib import font_manager, rc 
# from matplotlib import font_manager, rc

# font_fname = 'c:/windows/fonts/gulim.ttc'     # A font of your choice
# font_name = font_manager.FontProperties(fname=font_fname).get_name()
# rc('font', family=font_name)


# fig = plt.figure()
# fig.set_label('test')
# plt.figure(figsize=(20, 10)) 
# text.plot(50)

# 3.데이터 전처리 (Data Preprocessing)

##  5) 불용어 처리 ( Stop words remove)

In [ ]:
test = []
# stop_words=['우아하다']
for i in range(len(title_text)):
    j = 0
    while(j < len(title_text[i])):
        if(title_text[i][j] in stop_words):
            title_text[i].remove(title_text[i][j])
            continue
        j += 1
    test.append(title_text[i])

print(test)

In [ ]:
# def tokenize_new(doc):
#     x = [line.rstrip('\n') for line in open('C:/Users/DongA2/Final_Final_Disertation/File/new_stop.txt', 'r', encoding='utf-8')]
#     word_tokens = tagger.nouns(doc)
#     result = [] 
    
#     for w in word_tokens: 
#         if w not in x: 
            
#             result.append(w) 
#     return result

# 4.word2vec 벡터화

In [13]:
#word2vec 파라미터 설명
# workers =프로세스개수
# size= 워드 벡터 특징값수(벡터의 차원을 정한다.)
# min_count =단어에 대한 최소 빈도수(적은 빈도수의 단어는 학습하지 않는다. )
# window =컨테스트 윈도우 크기
# sample=샘플링 비율 (Ex)1e-3 :0.001이 좋은 성능을 낸다


In [ ]:
#진행상황확인위해
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)


In [ ]:
from gensim.models import Word2Vec
print("Training model...")
model= Word2Vec(title_text, size=300, window=5, min_count=5, workers=4, sg=1)

# 5.텍스트 유사도

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
train1, test1 = train_test_split(title, test_size=0.2)

In [ ]:
train1.info

## 명사만 추가한 리스트 만들기 (extraction only nouns)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

okt=Okt()

doc_nouns_list=[]

for doc in train_title:
    nouns=okt.nouns(doc)
    doc_nouns=''
    
    for noun in nouns:
        doc_nouns+=noun + ' '
    doc_nouns_list.append(doc_nouns)
    
for i in range(0,len(train_title)):
    print('doc' +str(i+1)+ ' : ' +str(doc_nouns_list[i]))

## 문장 유사도  (Text Similarity)

In [ ]:
tfidf_vectorizer=TfidfVectorizer(min_df=1)
tfidf_matrix = tfidf_vectorizer.fit_transform(doc_nouns_list)

document_distances=(tfidf_matrix*tfidf_matrix.T)

print('유사도 분석을 위한' + str(document_distances.get_shape()[0])+'x'+str(document_distances.get_shape()[1])+'matrix를 만들었습니다')
print(document_distances.toarray())    

## 6) 유사도 붙이기 

https://stackoverflow.com/questions/22129943/how-to-calculate-the-sentence-similarity-using-word2vec-model-of-gensim-with-pyt

In [ ]:
# s1_afv = avg_feature_vector('this is a sentence', model=model, num_features=300, index2word_set=index2word_set)
# s2_afv = avg_feature_vector('this is also sentence', model=model, num_features=300, index2word_set=index2word_set)
# sim = 1 - spatial.distance.cosine(s1_afv, s2_afv)
# print(sim)

# > 0.915479828613